Let $\Pi$ be the function ```unitary_to_monodromy```. Then for $XY(\alpha) = \exp(-i\alpha (XX+YY))$, we have $\Pi(XY(\alpha)) = \left( \frac{2\alpha}{\pi}, 0, 0, -\frac{2\alpha}{\pi}\right)$. Since $0 \leq \alpha \leq \frac{\pi}{4}$ by page $4$ of $XX$, we have that $0\leq\frac{2\alpha}{\pi} =x_1 \leq \frac{1}{2}$.

### Computing XY Polytope

In [68]:
import numpy as np
from fractions import *

from time import perf_counter

import sys

sys.path.insert(1, "/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy")

import monodromy

from monodromy.coordinates import monodromy_alcove, monodromy_alcove_c2, monodromy_to_positive_canonical_polytope, rho_reflect
from monodromy.elimination import cylinderize, project
from monodromy.polytopes import ConvexPolytope, Polytope
from monodromy.static import qlr_polytope

from itertools import count

def compute_polytope(n, angle):
    
    num = Fraction(angle).limit_denominator().numerator
    denom = Fraction(angle).limit_denominator().denominator

    biswas_relations = (qlr_polytope
        # enlarge to the pu_4 version of the QLR relations
        .union(rho_reflect(qlr_polytope, [0, 7, 8, 9]))
        # constrain in- and out-coordinates to the appropriate alcove
        .intersect(cylinderize(monodromy_alcove, [0, 1, 2, 3], 10))
        .intersect(cylinderize(monodromy_alcove_c2, [0, 7, 8, 9], 10))
    )

    """THIS IS IMPORTANT"""
    # constrain interaction coordinates to be of XY-type
    biswas_relations = biswas_relations.intersect(Polytope(convex_subpolytopes=[
        ConvexPolytope(
            inequalities=[[num, 0, 0, 0, -denom, 0, 0, 0, 0, 0]], # Angle set
            equalities=[
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],  # x2 == 0 # What kind of gate you want
                [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],  # x3 == 0 
            ]
        )
    ]))

    # switch to canonical coordinates
    biswas_relations = monodromy_to_positive_canonical_polytope(
        biswas_relations, coordinates=[0, 1, 2, 3])
    biswas_relations = monodromy_to_positive_canonical_polytope(
        biswas_relations, coordinates=[0, 4, 5, 6])
    biswas_relations = monodromy_to_positive_canonical_polytope(
        biswas_relations, coordinates=[0, 7, 8, 9])

    # reduce the biswas relations to have following coordinates:
    # k a1 a2 a3 beta b1 b2 b3
    biswas_relations = biswas_relations.reduce()
    biswas_relations = project(biswas_relations, 6).reduce()
    biswas_relations = project(biswas_relations, 5).reduce()

    xy_polytope = monodromy.static.examples.identity_polytope
    polytope_list = []
    for n in range(n+1):

        print(f"Working on an XY interaction sequence of length {n}...")

        # inflate xy_polytope from [*a_coords, *interaction_coords] to [*a_coords, *b_coords, *interaction_coords, beta]
        xy_polytope = cylinderize(
            xy_polytope,
            coordinate_map=[0, 1, 2, 3] + list(range(7, 7 + (n - 1))),
            parent_dimension=1 + 3 + 3 + n,
        ).intersect(cylinderize(
            biswas_relations,
            coordinate_map=[0, 1, 2, 3, -1, 4, 5, 6],
            parent_dimension=1 + 3 + 3 + n,
        ))

        # project away the old a-coordinates
        start_time = perf_counter()
        print("Working on the reduction 1/3...", end="")
        xy_polytope = project(xy_polytope, 3).reduce()
        print(f" done.  Took {perf_counter() - start_time} seconds.")

        start_time = perf_counter()
        print("Working on the reduction 2/3...", end="")
        xy_polytope = project(xy_polytope, 2).reduce()
        print(f" done.  Took {perf_counter() - start_time} seconds.")

        start_time = perf_counter()
        print("Working on the reduction 3/3...", end="")
        xy_polytope = project(xy_polytope, 1).reduce()
        print(f" done.  Took {perf_counter() - start_time} seconds.")

        # now the old c-coordinates are sitting where the a-coordinates were!
        print("The first three coordinates are the canonical coordinates CAN(x1, x2, x3).")
        print("The remaining coordinates x4, ..., xk are the XY interaction strengths.")
        print(xy_polytope)

        polytope_list.append(xy_polytope)
        
    return polytope_list

### Print Table

In [69]:
import os

def ineq_str(data, no_extra_rows, name):
    # Define the headers
    headers = ["1", "a_1", "a_2", "a_3"] + [f"𝛼_{i+1}" for i in range(no_extra_rows)]
    
    data = sorted(data)
    
    # Ensure that the data length matches the number of headers
    if not all(len(row) == len(headers) for row in data):
        raise ValueError("All rows in the data must have the same length as the headers.")
    
    # Calculate the maximum width for each column
    col_widths = [max(len(str(item)) for item in [header] + [row[i] for row in data]) for i, header in enumerate(headers)]
    
    # Define a function to print a single row with given data and column widths
    def row_str(row):
        return " | ".join(f"{str(item).ljust(col_widths[i])}" for i, item in enumerate(row))
    
    ineq_str_ = ""
    # Add name and number of inequalities
    ineq_str_ += f"{name} ({len(data)} inequalities)\n"

    # Add the headers
    ineq_str_ += row_str(headers) + "\n"
    ineq_str_ += "-+-".join("-" * width for width in col_widths) + "\n"
    
    # Add each row of data
    for row in data:
        ineq_str_ += row_str(row) + "\n"
        
    ineq_str_ += "\n"
    
    return ineq_str_

def classify(ineq_system, poly = 0, save=False, table_name=""):
    interaction_ineq, CAN_ineq, total_strength_ineq = [], [], []
    slant_ineq, height_ineq, other_ineq = [], [], []
    spade_ineq, club_ineq, diamond_ineq, heart_ineq = [], [], [], []
    
    if poly == 0:
        for ineq in ineq_system:
            if ineq[1:4] == [0, 0, 0]:
                interaction_ineq.append(ineq)
            elif ineq[4:] == [0 for _ in range(len(ineq[4:]))]:
                CAN_ineq.append(ineq)
            elif ineq[:4] == [0, -1, -1, -1]:
                total_strength_ineq.append(ineq)
            elif ineq[:4] == [0, 1, -1, -1]:
                slant_ineq.append(ineq)
            elif ineq[:4] == [0, 0, 0, -1]:
                height_ineq.append(ineq)
            elif ineq[0:4] == [0, -1, 0, 0]:
                spade_ineq.append(ineq)
            elif ineq[:4] == [0, -1, 1, -1]:
                club_ineq.append(ineq)
            elif ineq[:4] == [0, 0, 1, 0]:
                diamond_ineq.append(ineq)
            elif ineq[:4] == [0, 1, 1, -1]:
                heart_ineq.append(ineq)
            else:
                other_ineq.append(ineq)
    elif poly == 1:
        for ineq in ineq_system:
            if ineq[1:4] == [0, 0, 0]:
                interaction_ineq.append(ineq)
            elif ineq[4:] == [0 for _ in range(len(ineq[4:]))]:
                CAN_ineq.append(ineq)
            elif ineq[:4] == [-1, 1, -1, -1]:
                total_strength_ineq.append(ineq)
            elif ineq[:4] == [1, -1, -1, -1]:
                slant_ineq.append(ineq)
            elif ineq[:4] == [0, 0, 0, -1]:
                height_ineq.append(ineq)
            elif ineq[:4] == [-1, 1, 0, 0]:
                spade_ineq.append(ineq)
            elif ineq[:4] == [-1, 1, 1, -1]:
                club_ineq.append(ineq)
            elif ineq[:4] == [0, 0, 1, 0]:
                diamond_ineq.append(ineq)
            elif ineq[:4] == [1, -1, 1, -1]:
                heart_ineq.append(ineq)
            else:
                other_ineq.append(ineq)
    
    n = len(ineq_system[0]) - 4
    
    table_str = ""
    table_str += ineq_str(interaction_ineq, n, "Interaction Ineq")
    table_str += ineq_str(CAN_ineq, n, "CAN Ineq")
    table_str += ineq_str(total_strength_ineq, n, "Total Strength Ineq")
    table_str += ineq_str(slant_ineq, n, "Slant Ineq")
    table_str += ineq_str(height_ineq, n, "Height Ineq")
    table_str += ineq_str(spade_ineq, n, "Spade Ineq")
    table_str += ineq_str(club_ineq, n, "Club Ineq")
    table_str += ineq_str(diamond_ineq, n, "Diamond Ineq")
    table_str += ineq_str(heart_ineq, n, "Heart Ineq")
    table_str += ineq_str(other_ineq, n, "Other Ineq")
    
    if save:  # Save string to file
        # Ensure the file name ends with .txt
        if not table_name.endswith('.txt'):
            table_name += '.txt'    
        
        # Folder path
        folder_path = "/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/inequalities/"
        
        # Create the folder if it does not exist
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        # Create the full path to the file
        file_path = os.path.join(folder_path, table_name)
            
        # Check if file already exists
        if os.path.exists("inequalities/" + table_name):
            raise FileExistsError(f"The file {table_name} already exists.")

        # Open the file in write mode and save the string
        with open("inequalities/" + table_name, 'w') as file:
            file.write(table_str)

        print(f"String has been saved to {table_name}")
                
    return table_str

### Inequalities

Naming convention for polytope file: xy + angle + number of gates + which polytope (starting from 0). For example xypi8n3p0.txt. Note that $1$ in the $1$ column of the inequalities correspond to $\frac{\pi}{2}$.

In [86]:
polytope_list = [Polytope(convex_subpolytopes=[ConvexPolytope(inequalities=[[1, 0, 0, 0]], equalities=[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], name='origin ∩ anonymous_convex_polytope_7'), ConvexPolytope(inequalities=[[1, 0, 0, 0]], equalities=[[-1, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]], name='origin ∩ anonymous_convex_polytope_8')]), Polytope(convex_subpolytopes=[ConvexPolytope(inequalities=[[0, 1, 0, 0, 0], [1, -4, 0, 0, 0]], equalities=[[0, -1, 1, 0, 0], [0, 0, 0, 1, 0], [0, -1, 0, 0, 1]], name='origin ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7'), ConvexPolytope(inequalities=[[1, -1, 0, 0, 0], [-3, 4, 0, 0, 0]], equalities=[[-1, 1, 1, 0, 0], [0, 0, 0, 1, 0], [-1, 1, 0, 0, 1]], name='origin ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_8')]), Polytope(convex_subpolytopes=[ConvexPolytope(inequalities=[[0, -1, 1, -1, 2, 0], [0, 0, 1, -1, 0, 0], [0, 1, -1, -1, 0, 0], [1, 0, 0, 0, 0, -4], [0, -1, 0, 0, 1, 1], [0, -1, 1, -1, 0, 2], [1, 0, 0, 0, -4, 0], [0, 0, 0, 1, 0, 0], [0, 0, 1, 0, -1, 1], [0, 0, 1, 0, 1, -1]], equalities=[], name='origin ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7'), ConvexPolytope(inequalities=[[0, 0, 0, 1, 0, 0], [1, -1, -1, -1, 0, 0], [0, 0, 1, 0, 1, -1], [1, 0, 0, 0, 0, -4], [0, 0, 1, -1, 0, 0], [1, 0, 0, 0, -4, 0], [0, 0, 1, 0, -1, 1], [-1, 1, 1, -1, 0, 2], [-1, 1, 0, 0, 1, 1], [-1, 1, 1, -1, 2, 0]], equalities=[], name='origin ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_8')]), Polytope(convex_subpolytopes=[ConvexPolytope(inequalities=[[0, 0, 1, -1, 0, 0, 0], [0, 1, 1, -1, -2, 2, 2], [0, 0, 1, 0, -1, 1, 1], [0, 1, -1, 0, 0, 0, 0], [0, 0, 1, 0, 1, -1, 1], [0, 0, 0, 0, 0, 0, 1], [0, 1, 1, -1, 2, -2, 2], [1, 0, 0, 0, 0, -4, 0], [1, 0, 0, 0, -4, 0, 0], [0, 0, 0, -1, 0, 1, 1], [0, 1, -1, -1, 0, 0, 2], [0, 1, -1, -1, 0, 2, 0], [0, 1, -1, -1, 2, 0, 0], [0, 1, 1, -1, 2, 2, -2], [1, 0, 0, 0, 0, 0, -4], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 1, 1, -1], [0, 0, 0, 0, 0, 1, 0], [1, -1, -1, 0, 0, 0, 0], [0, -1, -1, -1, 2, 2, 2], [0, 0, 0, -1, 1, 0, 1], [0, 0, 0, -1, 1, 1, 0], [0, -1, 0, 0, 1, 1, 1], [0, -1, 1, -1, 2, 0, 2], [0, -1, 1, -1, 2, 2, 0], [0, -1, 1, -1, 0, 2, 2]], equalities=[], name='origin ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7'), ConvexPolytope(inequalities=[[0, 0, 0, 1, 0, 0, 0], [0, 0, 1, -1, 0, 0, 0], [1, 0, 0, 0, 0, 0, -4], [0, 0, 1, 0, 1, 1, -1], [1, 0, 0, 0, -4, 0, 0], [0, 0, 1, 0, -1, 1, 1], [1, -1, 1, -1, -2, 2, 2], [0, 0, 0, 0, 0, 1, 0], [1, -1, 1, -1, 2, 2, -2], [1, -1, -1, 0, 0, 0, 0], [1, -1, -1, -1, 0, 2, 0], [-1, 1, 0, 0, 1, 1, 1], [-1, 1, 1, -1, 2, 2, 0], [-1, 1, -1, -1, 2, 2, 2], [0, 1, -1, 0, 0, 0, 0], [0, 0, 0, -1, 1, 0, 1], [-1, 1, 1, -1, 2, 0, 2], [1, -1, 1, -1, 2, -2, 2], [0, 0, 1, 0, 1, -1, 1], [0, 0, 0, -1, 1, 1, 0], [1, -1, -1, -1, 2, 0, 0], [0, 0, 0, 0, 1, 0, 0], [-1, 1, 1, -1, 0, 2, 2], [0, 0, 0, -1, 0, 1, 1], [1, -1, -1, -1, 0, 0, 2], [0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, -4, 0]], equalities=[], name='origin ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_8 ∩ anonymous_convex_polytope_7')]), Polytope(convex_subpolytopes=[ConvexPolytope(inequalities=[[0, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 0, 0, -4], [0, 0, 0, 0, 0, 0, 1, 0], [0, -1, 1, -1, 2, 2, 2, 0], [0, -1, -1, -1, 2, 2, 2, 2], [0, -1, 0, 0, 1, 1, 1, 1], [0, -1, 1, -1, 0, 2, 2, 2], [0, 0, 0, -1, 0, 1, 1, 1], [0, 1, -1, -1, 2, 0, 2, 0], [0, 0, 0, -1, 1, 1, 0, 1], [0, 1, -1, -1, 0, 2, 2, 0], [0, 0, 1, -1, 0, 0, 0, 0], [0, 1, -1, -1, 0, 0, 2, 2], [0, 1, -1, -1, 0, 2, 0, 2], [1, 0, 0, 0, 0, 0, -4, 0], [0, -1, 1, -1, 2, 2, 0, 2], [1, 0, 0, 0, -4, 0, 0, 0], [0, 0, 1, 0, -1, 1, 1, 1], [0, 1, -1, 0, 0, 0, 0, 0], [0, 1, -1, -1, 2, 2, 0, 0], [0, 1, 1, -1, 2, 2, -2, 2], [0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 1, 0, 1, 1, -1, 1], [0, 1, -1, -1, 2, 0, 0, 2], [0, -1, 1, -1, 2, 0, 2, 2], [0, 1, 1, -1, 2, -2, 2, 2], [0, 0, 0, -1, 1, 0, 1, 1], [1, 0, 0, 0, 0, -4, 0, 0], [1, -1, -1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, -1, 1, 1], [0, 1, 1, -1, -2, 2, 2, 2], [0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 1, 0, 1, 1, 1, -1], [0, 1, 1, -1, 2, 2, 2, -2], [0, 0, 0, -1, 1, 1, 1, 0]], equalities=[], name='origin ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7'), ConvexPolytope(inequalities=[[1, -1, -1, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 1, 1, 1, -1], [0, 0, 0, 0, 1, 0, 0, 0], [1, -1, 1, -1, 2, 2, 2, -2], [1, -1, -1, -1, 2, 0, 2, 0], [1, -1, 1, -1, 2, 2, -2, 2], [1, -1, -1, -1, 2, 0, 0, 2], [1, -1, 1, -1, 2, -2, 2, 2], [1, -1, 1, -1, -2, 2, 2, 2], [0, 0, 1, 0, 1, 1, -1, 1], [0, 0, 1, 0, 1, -1, 1, 1], [0, 0, 1, 0, -1, 1, 1, 1], [1, 0, 0, 0, 0, 0, -4, 0], [0, 1, -1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1], [-1, 1, 1, -1, 2, 2, 0, 2], [1, 0, 0, 0, 0, -4, 0, 0], [0, 0, 0, 0, 0, 0, 1, 0], [-1, 1, 1, -1, 2, 0, 2, 2], [-1, 1, 0, 0, 1, 1, 1, 1], [-1, 1, 1, -1, 0, 2, 2, 2], [-1, 1, -1, -1, 2, 2, 2, 2], [1, 0, 0, 0, 0, 0, 0, -4], [0, 0, 1, -1, 0, 0, 0, 0], [-1, 1, 1, -1, 2, 2, 2, 0], [1, 0, 0, 0, -4, 0, 0, 0], [0, 0, 0, -1, 1, 1, 1, 0], [0, 0, 0, -1, 1, 1, 0, 1], [0, 0, 0, -1, 0, 1, 1, 1], [0, 0, 0, -1, 1, 0, 1, 1], [1, -1, -1, -1, 0, 2, 2, 0], [1, -1, -1, -1, 0, 0, 2, 2], [1, -1, -1, -1, 0, 2, 0, 2], [0, 0, 0, 0, 0, 1, 0, 0], [1, -1, -1, -1, 2, 2, 0, 0]], equalities=[], name='origin ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_8 ∩ anonymous_convex_polytope_7 ∩ anonymous_convex_polytope_7')])]

In [78]:
file = open("inequalities/XY pi:2.txt", "r")

polytope_list = eval(file.read()[-16687:])

In [80]:
P2 = polytope_list[3].convex_subpolytopes

In [52]:
massive_poly_list = [compute_polytope(3, 1/4), compute_polytope(3, 1/8), compute_polytope(3, 1/5)]

Working on an XY interaction sequence of length 0...
Working on the reduction 1/3... done.  Took 0.12907327502034605 seconds.
Working on the reduction 2/3... done.  Took 0.15493852295912802 seconds.
Working on the reduction 3/3... done.  Took 0.15954431612044573 seconds.
The first three coordinates are the canonical coordinates CAN(x1, x2, x3).
The remaining coordinates x4, ..., xk are the XY interaction strengths.
[
# origin ∩ anonymous_convex_polytope_47: 
    1 +     0 x1 +     0 x2 +     0 x3 >= 0
    0 +     1 x1 +     0 x2 +     0 x3 == 0
    0 +     0 x1 +     1 x2 +     0 x3 == 0
    0 +     0 x1 +     0 x2 +     1 x3 == 0
,
# origin ∩ anonymous_convex_polytope_48: 
    1 +     0 x1 +     0 x2 +     0 x3 >= 0
   -1 +     1 x1 +     0 x2 +     0 x3 == 0
    0 +     0 x1 +     1 x2 +     0 x3 == 0
    0 +     0 x1 +     0 x2 +     1 x3 == 0

]
Working on an XY interaction sequence of length 1...
Working on the reduction 1/3... done.  Took 0.2605264831800014 seconds.
Working on th

Working on an XY interaction sequence of length 0...
Working on the reduction 1/3... done.  Took 0.1360389159526676 seconds.
Working on the reduction 2/3... done.  Took 0.11306752101518214 seconds.
Working on the reduction 3/3... done.  Took 0.13330951402895153 seconds.
The first three coordinates are the canonical coordinates CAN(x1, x2, x3).
The remaining coordinates x4, ..., xk are the XY interaction strengths.
[
# origin ∩ anonymous_convex_polytope_55: 
    1 +     0 x1 +     0 x2 +     0 x3 >= 0
    0 +     1 x1 +     0 x2 +     0 x3 == 0
    0 +     0 x1 +     1 x2 +     0 x3 == 0
    0 +     0 x1 +     0 x2 +     1 x3 == 0
,
# origin ∩ anonymous_convex_polytope_56: 
    1 +     0 x1 +     0 x2 +     0 x3 >= 0
   -1 +     1 x1 +     0 x2 +     0 x3 == 0
    0 +     0 x1 +     1 x2 +     0 x3 == 0
    0 +     0 x1 +     0 x2 +     1 x3 == 0

]
Working on an XY interaction sequence of length 1...
Working on the reduction 1/3... done.  Took 0.3024278460070491 seconds.
Working on the

Working on an XY interaction sequence of length 0...
Working on the reduction 1/3... done.  Took 0.13165208720602095 seconds.
Working on the reduction 2/3... done.  Took 0.12435110402293503 seconds.
Working on the reduction 3/3... done.  Took 0.12194990203715861 seconds.
The first three coordinates are the canonical coordinates CAN(x1, x2, x3).
The remaining coordinates x4, ..., xk are the XY interaction strengths.
[
# origin ∩ anonymous_convex_polytope_63: 
    1 +     0 x1 +     0 x2 +     0 x3 >= 0
    0 +     1 x1 +     0 x2 +     0 x3 == 0
    0 +     0 x1 +     1 x2 +     0 x3 == 0
    0 +     0 x1 +     0 x2 +     1 x3 == 0
,
# origin ∩ anonymous_convex_polytope_64: 
    1 +     0 x1 +     0 x2 +     0 x3 >= 0
   -1 +     1 x1 +     0 x2 +     0 x3 == 0
    0 +     0 x1 +     1 x2 +     0 x3 == 0
    0 +     0 x1 +     0 x2 +     1 x3 == 0

]
Working on an XY interaction sequence of length 1...
Working on the reduction 1/3... done.  Took 0.2633068731520325 seconds.
Working on th

In [75]:
P32 = compute_polytope(3, 1/32)[3].convex_subpolytopes

Working on an XY interaction sequence of length 0...
Working on the reduction 1/3... done.  Took 0.16025489498861134 seconds.
Working on the reduction 2/3... done.  Took 0.1261688929516822 seconds.
Working on the reduction 3/3... done.  Took 0.10564858303405344 seconds.
The first three coordinates are the canonical coordinates CAN(x1, x2, x3).
The remaining coordinates x4, ..., xk are the XY interaction strengths.
[
# origin ∩ anonymous_convex_polytope_79: 
    1 +     0 x1 +     0 x2 +     0 x3 >= 0
    0 +     1 x1 +     0 x2 +     0 x3 == 0
    0 +     0 x1 +     1 x2 +     0 x3 == 0
    0 +     0 x1 +     0 x2 +     1 x3 == 0
,
# origin ∩ anonymous_convex_polytope_80: 
    1 +     0 x1 +     0 x2 +     0 x3 >= 0
   -1 +     1 x1 +     0 x2 +     0 x3 == 0
    0 +     0 x1 +     1 x2 +     0 x3 == 0
    0 +     0 x1 +     0 x2 +     1 x3 == 0

]
Working on an XY interaction sequence of length 1...
Working on the reduction 1/3... done.  Took 0.26812828495167196 seconds.
Working on th

In [58]:
P4 = massive_poly_list[0][3].convex_subpolytopes
P8 = massive_poly_list[1][3].convex_subpolytopes
P5 = massive_poly_list[2][3].convex_subpolytopes

In [76]:
print(float(P2[0].volume.volume), float(P4[1].volume.volume))
print(float(P4[0].volume.volume), float(P4[1].volume.volume))
print(float(P8[0].volume.volume), float(P8[1].volume.volume))
print(float(P5[0].volume.volume), float(P5[1].volume.volume))
print(float(P16[0].volume.volume), float(P16[1].volume.volume))
print(float(P32[0].volume.volume), float(P32[1].volume.volume))

0.00013309682838220165 0.00013309682838220165
2.1219253540039063e-06 2.1219253540039063e-06
3.55888888888889e-05 3.55888888888889e-05
3.3155083656311036e-08 3.3155083656311036e-08
5.180481821298599e-10 5.180481821298599e-10


In [71]:
0.00013309682838220165/2.1219253540039063e-06

62.72455726637999

In [74]:
2.1219253540039063e-06 / 3.3155083656311036e-08

64.0

In [77]:
3.3155083656311036e-08/ 5.180481821298599e-10

64.0

In [60]:
len(P8)

2

In [42]:
float(p0.volume.volume)

2.1219253540039063e-06

In [43]:
float(p0.volume.volume)

2.1219253540039063e-06

In [21]:
file = open("inequalities/XY pi:2.txt", "r")

polytope_list = eval(file.read()[-16687:])

In [24]:
polytope = polytope_list[4].convex_subpolytopes[5].inequalities
print(classify(polytope, poly = 0, save= False, table_name = "xypi4n4p2"))

Interaction Ineq (7 inequalities)
1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3 | 𝛼_4
--+-----+-----+-----+-----+-----+-----+----
0 | 0   | 0   | 0   | 0   | 0   | 1   | 0  
0 | 0   | 0   | 0   | 0   | 1   | 0   | 0  
0 | 0   | 0   | 0   | 1   | 0   | 0   | 0  
1 | 0   | 0   | 0   | -2  | 0   | 0   | 0  
1 | 0   | 0   | 0   | 0   | -2  | 0   | 0  
1 | 0   | 0   | 0   | 0   | 0   | -2  | 0  
1 | 0   | 0   | 0   | 0   | 0   | 0   | -2 

CAN Ineq (4 inequalities)
1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3 | 𝛼_4
--+-----+-----+-----+-----+-----+-----+----
0 | 0   | 0   | 1   | 0   | 0   | 0   | 0  
0 | 0   | 1   | -1  | 0   | 0   | 0   | 0  
0 | 1   | -1  | 0   | 0   | 0   | 0   | 0  
1 | -1  | -1  | 0   | 0   | 0   | 0   | 0  

Total Strength Ineq (0 inequalities)
1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3 | 𝛼_4
--+-----+-----+-----+-----+-----+-----+----

Slant Ineq (3 inequalities)
1 | a_1 | a_2 | a_3 | 𝛼_1 | 𝛼_2 | 𝛼_3 | 𝛼_4
--+-----+-----+-----+-----+-----+-----+----
0 | 1   | -1  | -1  | 0   | 2   | 2 

In [90]:
from scipy.optimize import linprog

c = np.array([[-1/2, 1/2, -1/2]])
A_ub = np.array([[-1, 1, 0], 
                 [0, -1, 1], 
                 [0, 0, -1], 
                 [1, 1, 0]])
b_ub = np.array([[0, 0, 0, 1/2]])

linprog(c, A_ub, b_ub)

     con: array([], dtype=float64)
     fun: -0.2500000013982017
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([ 5.00000003e-01,  4.86177269e-10,  1.45059839e-10, -3.91381771e-09])
  status: 0
 success: True
       x: array([5.00000003e-01, 6.31237108e-10, 1.45059839e-10])